In [ ]:
#Todos os imports necessários
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from scipy import stats
from sklearn.ensemble import RandomForestRegressor


In [ ]:
# Importando o dataset de treino
base = pd.read_csv('train.csv')

In [ ]:
# Eliminando colunas com mais de 10% de valores vazios
eliminar = base.columns[(base.isnull().sum() / base.shape[0]) > 0.1]
base = base.drop(eliminar, axis=1)

# Selecionando apenas as colunas numéricas
colunas = base.columns[base.dtypes != 'object']
base2 = base.loc[:, colunas]
base2

In [ ]:
# Substituindo os valores vazios por -1
base2 = base2.fillna(-1)
base2

In [ ]:
# Removendo outliers usando Z-score
z_scores = np.abs(stats.zscore(base2))
threshold = 3
base2 = base2[(z_scores < threshold).all(axis=1)]

In [ ]:
# Criando novas features
base2['TotalSF'] = base2['TotalBsmtSF'] + base2['1stFlrSF'] + base2['2ndFlrSF']

In [ ]:
# Selecionando X e y
X = base2.drop(['SalePrice'], axis=1)
y = base2.SalePrice

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# Padronizando os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Usando um modelo de Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Avaliando o erro da regressão
print(f'Mean Squared Error: {mean_squared_error(y_test, y_pred)}')

In [ ]:
# Validação cruzada
cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
print(f'Cross-Validated MSE: {-cv_scores.mean()}')

In [ ]:
# Importando a base de teste
teste = pd.read_csv('test.csv')

# Eliminando as mesmas colunas da base de treino
teste = teste.drop(eliminar, axis=1)

# Selecionando apenas as colunas numéricas
colunas2 = teste.columns[teste.dtypes != 'object']
teste = teste.loc[:, colunas2]

# Substituindo os valores vazios por -1
teste = teste.fillna(-1)

# Criando novas features para o teste
teste['TotalSF'] = teste['TotalBsmtSF'] + teste['1stFlrSF'] + teste['2ndFlrSF']

# Padronizando a base de teste
teste_scaled = scaler.transform(teste)

# Fazendo a previsão para a base de teste
y_pred_test = model.predict(teste_scaled)

# Adicionando a coluna de previsão na base de teste
teste['SalePrice'] = y_pred_test

# Extraindo apenas as colunas Id e SalePrice
resultado = teste[['Id', 'SalePrice']]
print(resultado.head(3))
# Podemos então exportar essa base
resultado.to_csv('resultado.csv', index=False)